In [ ]:
!pip install opencv-python

In [ ]:
!pip install mediapipe --upgrade --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [ ]:
import numpy as np
import cv2
import mediapipe as mp
import time
import pandas as pd
import os

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="bmnaJcci8oanfPymxPhF")
project = rf.workspace("roboflow-58fyf").project("rock-paper-scissors-sxsw")
version = project.version(14)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to rock-paper-scissors-14 in yolov8:: 100%|██████████| 14682/14682 [00:01<00:00, 7774.90it/s] 


In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=True, max_num_hands= 1)

In [ ]:
import os
import cv2
import mediapipe as mp

data = []
labels = []

images_folder = "/content/rock-paper-scissors-14/train/images"
labels_folder = "/content/rock-paper-scissors-14/train/labels"

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True)

for img_file in os.listdir(images_folder):
    if not img_file.endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(images_folder, img_file)
    label_file = os.path.splitext(img_file)[0] + ".txt"
    label_path = os.path.join(labels_folder, label_file)

    # Check if label file exists

    if not os.path.exists(label_path):
        continue

    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmarks = []
        for lm in hand_landmarks.landmark:
            landmarks.append(lm.x)
            landmarks.append(lm.y)

        # Read the label only if landmarks were extracted successfully

        with open(label_path, "r") as f:
            lines = f.readlines()
            if lines:
                class_id = int(lines[0].split()[0])
                data.append(landmarks)
                labels.append(class_id)


hands.close()



In [ ]:
labels[:5]

[2, 2, 1, 0, 2]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X = np.array(data)
y = np.array(labels)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))






Accuracy: 0.8452830188679246
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.85       161
           1       0.86      0.85      0.85       210
           2       0.87      0.80      0.83       159

    accuracy                           0.85       530
   macro avg       0.85      0.84      0.84       530
weighted avg       0.85      0.85      0.85       530



In [ ]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

video_path = "/content/drive/MyDrive/WhatsApp Video 2025-04-14 at 03.04.09_c242159c.mp4"

cap = cv2.VideoCapture(video_path)

# Get video properties

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_with_prediction.mp4', fourcc, fps, (width, height))



def get_class_label(prediction):
    label_mapping = {0: "Paper", 1: "Rock", 2: "Scissors"}
    return label_mapping.get(prediction, "Unknown")


while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            if idx < 2:  # Process only the first two hands (0, 1)
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.append(lm.x)
                    landmarks.append(lm.y)

                if len(landmarks) == X.shape[1]:
                    prediction = model.predict([landmarks])  # Predict the class
                    label = get_class_label(prediction[0])  # Map the prediction to the label
                else:
                    label = "Unknown"

                # Draw landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get position of the first landmark to display the text
                cx = int(hand_landmarks.landmark[0].x * width)
                cy = int(hand_landmarks.landmark[0].y * height)

                # Display the class prediction on the frame
                cv2.putText(frame, f"Hand #{idx+1}: {label}", (cx, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # show individual landmark coordinates

                for i, lm in enumerate(hand_landmarks.landmark[:3]):  # Display top 3 landmarks
                    coord_text = f"{i}: ({lm.x:.2f}, {lm.y:.2f})"
                    cv2.putText(frame, coord_text, (cx, cy + 20 + i * 20),
                                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 1)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(" output_with_prediction.mp4")


 output_with_prediction.mp4
